In [27]:
import pandas as pd
import time 
import requests
import datetime 
from bs4 import BeautifulSoup
import smtplib
import re
import warnings
warnings.filterwarnings("ignore")

In [34]:
urls=[
    "https://www.kenyabizlist.com/category/special-needs-schools",
]
pattern=r'<a href="(?P<website>https://www.kenyabizlist.com/business/\d+[-\w]+)">.*?<b>(?P<name>.*?)</b></a>.*?P\.O\. Box (?P<location>[\d\s\w,]+)'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
data=[]
for URL in urls:
    page=requests.get(URL)
    soup=BeautifulSoup(page.text,"html")
    soup_find = soup.find_all(class_="c-detail")
    for element in soup_find:
        
        element_str = str(element)
        
        matches = re.finditer(pattern, element_str, re.DOTALL)
        
        
        # Extract and print each match
        for match in matches:
            website = match.group("website")
            name = match.group("name").strip()
            location = match.group("location").strip()
            
            # print(f"Website: {website}")
            # print(f"Name: {name}")
            # print(f"Location (P.O. Box): {location}")
            
            business_page = requests.get(website, headers=headers)
            business_soup = BeautifulSoup(business_page.text, "html.parser")
            
           
            try:
                phone = business_soup.find(text=re.compile(r'Phone')).find_next().get_text(strip=True)
            except AttributeError:
                phone = "N/A"
            
            try:
                email = business_soup.find(text=re.compile(r'Email')).find_next().get_text(strip=True)
            except AttributeError:
                email = "N/A"
            
            # Add the scraped data to the list
            data.append({
                "Website": website,
                "Name": name,
                "Location (P.O. Box)": location,
                "Phone": phone,
                "Email": email
            })

df = pd.DataFrame(data)
df.to_csv("special_needs_schools_data12.csv", index=False)

print("Data saved to special_needs_schools_data.csv")

Data saved to special_needs_schools_data.csv


In [35]:
for web in df["Website"]:
    print(f"Scraping data for: {web}")  # Print to confirm which website is being processed
    
    page = requests.get(web)
    
    
    if page.status_code == 200:
        # Parse the page content using BeautifulSoup
        soup = BeautifulSoup(page.text, "html.parser")
        
        # Find the specific data you're looking for on the page
        # Example: Scraping the 'c-info' class (assuming it holds contact details)
        soup_find = soup.find_all(class_="c-info")
        
        # Check if data exists in 'soup_find' and print or store the results
        for element in soup_find:
            print(element.get_text(strip=True))  # Extract and print the text for each 'c-info' element
    else:
        print(f"Failed to retrieve data for: {web} (Status code: {page.status_code})")

Scraping data for: https://www.kenyabizlist.com/business/2238-siuna-community-childrens-home
Address:Opposite Kitale AirportP.O. Box 2985, Kitale 30200City/Town:KitaleTelephone Number:0722296151Telephone Number 2:0722850117Email Address:Website:Category:Special Needs Schools
Scraping data for: https://www.kenyabizlist.com/business/2236-homa-bay-childrens-home-academy
Address:Next to SDA ChurchP.O. Box 77, Homa Bay  40300City/Town:Homa BayTelephone Number:05922312Email Address:Website:Category:Special Needs Schools
Scraping data for: https://www.kenyabizlist.com/business/2232-riruta-shade-for-the-orphaned-children
Address:Riruta Setellite Shopping CentreOpp Police StationP.O. Box 34602, Nairobi 00100City/Town:RirutaTelephone Number:0718119481Telephone Number 2:0721693409Email Address:Website:http://www.rirutashadeorphans.orgCategory:Special Needs Schools
Scraping data for: https://www.kenyabizlist.com/business/2231-mombasa-secondary-school-for-the-physically-handicapped
Address:Mombasa-

In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Read the CSV file containing the websites
df = pd.read_csv("special_needs_schools_data12.csv")

# Lists to hold the extracted data
addresses = []
towns = []
telephone_numbers_1 = []
telephone_numbers_2 = []
emails = []
websites = []
products = []  # New list for products

# Define a pattern to extract the required information
address_pattern = r'Address:(.*?)(?=City/Town:|Telephone Number:|$)'
town_pattern = r'City/Town:(.*?)(?=Telephone Number:|$)'
telephone_1_pattern = r'Telephone Number:(.*?)(?=Telephone Number 2:|Email Address:|$)'
telephone_2_pattern = r'Telephone Number 2:(.*?)(?=Email Address:|Website:|$)'
email_pattern = r'Email Address:(.*?)(?=Website:|Category:|$)'
website_pattern = r'Website:(.*?)(?=Category:|$)'

# Loop over each website in the 'Website' column
for web in df["Website"]:
    print(f"Scraping data for: {web}")  # Print to confirm which website is being processed
    
    page = requests.get(web)
    
    if page.status_code == 200:
        # Parse the page content using BeautifulSoup
        soup = BeautifulSoup(page.text, "html.parser")
        
        # Find the specific data you're looking for on the page
        soup_find = soup.find_all(class_="c-info")
        
        # Combine all text content into one string for regex processing
        combined_text = ' '.join([element.get_text(strip=True) for element in soup_find])
        
        # Extracting data using regular expressions
        address = re.search(address_pattern, combined_text)
        town = re.search(town_pattern, combined_text)
        telephone_1 = re.search(telephone_1_pattern, combined_text)
        telephone_2 = re.search(telephone_2_pattern, combined_text)
        email = re.search(email_pattern, combined_text)
        website = re.search(website_pattern, combined_text)

        # Extract product information from the span with ID 'lblProducts'
        # product_info = soup.find_all(id="lblProducts")
        # products.append(product_info.get_text(separator=", ", strip=True) if product_info else "Not found")

        # Append extracted data to lists, or "Not found" if not found
        addresses.append(address.group(1).strip() if address else "Not found")
        towns.append(town.group(1).strip() if town else "Not found")
        telephone_numbers_1.append(telephone_1.group(1).strip() if telephone_1 else "Not found")
        telephone_numbers_2.append(telephone_2.group(1).strip() if telephone_2 else "Not found")
        emails.append(email.group(1).strip() if email else "Not found")
        websites.append(website.group(1).strip() if website else "Not found")
    else:
        print(f"Failed to retrieve data for: {web} (Status code: {page.status_code})")
        addresses.append(None)
        towns.append(None)
        telephone_numbers_1.append(None)
        telephone_numbers_2.append(None)
        emails.append(None)
        websites.append(None)
        products.append(None)

# Add the new data to the existing DataFrame
df['Address'] = addresses
df['City/Town'] = towns
df['Telephone Number 1'] = telephone_numbers_1
df['Telephone Number 2'] = telephone_numbers_2
df['Email Address'] = emails
df['Website'] = websites
# df['Products'] = products  # Add the new column for product data

# Save the updated DataFrame to a new CSV
df.to_csv("updated_special_needs_schools_data.csv", index=False)

print("Data scraping and saving to CSV completed successfully!")


Scraping data for: https://www.kenyabizlist.com/business/2238-siuna-community-childrens-home
Scraping data for: https://www.kenyabizlist.com/business/2236-homa-bay-childrens-home-academy
Scraping data for: https://www.kenyabizlist.com/business/2232-riruta-shade-for-the-orphaned-children
Scraping data for: https://www.kenyabizlist.com/business/2231-mombasa-secondary-school-for-the-physically-handicapped
Scraping data for: https://www.kenyabizlist.com/business/2227-chekombero-special-school-for-the-deaf
Scraping data for: https://www.kenyabizlist.com/business/2225-lisa-hola-school-for-the-deaf
Scraping data for: https://www.kenyabizlist.com/business/2224-tumu-tumu-school-for-the-deaf
Scraping data for: https://www.kenyabizlist.com/business/2223-kuja-special-secondary-school-for-the-deaf
Scraping data for: https://www.kenyabizlist.com/business/2219-model-center-for-deaf-education--training
Scraping data for: https://www.kenyabizlist.com/business/2218-iten-school-for-the-deaf
Scraping data

In [47]:
df.head()
print(df.shape)

(20, 10)


In [48]:
df.value_counts()

Series([], Name: count, dtype: int64)

In [49]:
duplicates=df.duplicated()
duplicates.sum()

0

In [50]:
if duplicates.any():
    print("Duplicated rows:")
    print(df[duplicates])

In [51]:
df_cleaned=df.drop_duplicates()

In [52]:
df_cleaned.to_csv("cleaned_special_needs_schools_data.csv",index=False)


In [53]:
df_cleaned.shape

(20, 10)

In [45]:
df["Email"].value_counts()

Email
mssph014@gmail.com                     1
tumutumuschdeaf@yahoo.com              1
kttid2007@yahoo.com                    1
ienfo@deafchildrensociety-kenya.org    1
st.francis@yahoo.com                   1
highschoolucy@gmail.com                1
kblind@keib.or.ke                      1
kco@sightsavers.or.ke                  1
Name: count, dtype: int64

In [ ]:
url2=[
    "https://www.kenyabizlist.com/category/special-needs-schools?start=30"
]
pattern=r'<a href="(?P<website>https://www.kenyabizlist.com/business/\d+[-\w]+)">.*?<b>(?P<name>.*?)</b></a>.*?P\.O\. Box (?P<location>[\d\s\w,]+)'
for URL in url2:
    page=requests.get(URL)
    soup=BeautifulSoup(page.text,"html")
    soup_find = soup.find_all(class_="c-detail")
    for element in soup_find:
        
        element_str = str(element)
        
        matches = re.finditer(pattern, element_str, re.DOTALL)
        
        
        # Extract and print each match
        for match in matches:
            website = match.group("website")
            name = match.group("name").strip()
            location = match.group("location").strip()
            
            print(f"Website: {website}")
            print(f"Name: {name}")
            print(f"Location (P.O. Box): {location}")

In [ ]:
# soup_find=soup.find_all(class_="c-detail")

In [25]:
# product data pulling 
website=df["Website"][5]

In [26]:
website

'https://www.kenyabizlist.com/business/2225-lisa-hola-school-for-the-deaf'